In [ ]:
import sys
from pathlib import Path

REPO_ROOT = Path(r"C:\ML-Malware").resolve()
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

from malrob.family_evasion import (
    FamilyEvasionConfig,
    load_fam_mal,
    select_top_families,
    build_family_evasion_table,
    save_family_outputs,
    compute_family_delta_table,
    rank_families_by_risk,
    plot_family_curves,
    plot_combined_attacks_panel,
    export_latex_tables,
)

# ----------------------------
# Dynamic knobs (change only these)
# ----------------------------
TARGET_PER_CLASS = 6000
DATASET_TAG = f"{(TARGET_PER_CLASS * 2)//1000}k"
MODELS_DIR = REPO_ROOT / f"models_full_{DATASET_TAG}"

RESULTS_DIR = MODELS_DIR / "results_adv"
ARTIFACTS_DIR = MODELS_DIR / "artifacts_adv"

cfg = FamilyEvasionConfig(
    eps_list=(0.01, 0.05, 0.10),
    attacks=("FGSM", "PGD"),
    top_k=5,
    exclude_families=("unknown",),
    strict_alignment=True,
)

fam_mal = load_fam_mal(ARTIFACTS_DIR)
top_fams = select_top_families(fam_mal, top_k=cfg.top_k, exclude=cfg.exclude_families)
print("Top families:", top_fams)

df_fam = build_family_evasion_table(artifacts_dir=ARTIFACTS_DIR, cfg=cfg, top_families=top_fams)
save_family_outputs(df_fam=df_fam, results_dir=RESULTS_DIR)

df_delta = compute_family_delta_table(df_fam)
df_risk = rank_families_by_risk(df_delta, agg="mean")

print(df_risk.head(10))

plot_family_curves(df_fam=df_fam, results_dir=RESULTS_DIR, top_families=top_fams, attacks=cfg.attacks)

# Optional: single combined poster plot
plot_combined_attacks_panel(df_fam=df_fam, results_dir=RESULTS_DIR, top_families=top_fams)

# Optional: LaTeX tables
export_latex_tables(df_delta=df_delta, df_risk=df_risk, results_dir=RESULTS_DIR)

print("[DONE] NOTEBOOK 04 COMPLETED")
